In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  2 13:41:48 2023

@author: localadmin
"""
import pandas as pd
import os
import numpy as np
from pathlib import Path
import time

path_ECG_Valid = r'/mnt/r/ENGR_Chon/NIH_Pulsewatch_Database/Parquet_Files/Solo_ECG_Valid_each_UID'
path_ECG_Timestamp = r'/mnt/r/ENGR_Chon/NIH_Pulsewatch_Database/Parquet_Files/Solo_ECG_Timestamp_each_UID'
path_output = r'/mnt/r/ENGR_Chon/NIH_Pulsewatch_Database/Parquet_Files/Solo_ECG_Invalid_count_time'
Path(path_output).mkdir(parents=True, exist_ok=True)

fs_ECG = 250

# Get all the UIDs that were generated by MATLAB.
get_UID_list = r'/mnt/r/ENGR_Chon/NIH_Pulsewatch_Database/mat_for_load_Final_Clinical_Trial_Data'
dir_list_UID = os.listdir(get_UID_list)
first_3_char = list(set([x[:3] for x in dir_list_UID]))
first_3_char_unique = [x for x in first_3_char if x[0] == '0' or x[0] == '1']
first_3_char_unique.sort()

## For loop for each UID.
temp_UIDs = first_3_char_unique[-2:]
print('temp_UIDs:',temp_UIDs)

for UID in temp_UIDs:
    filename_ECG_Valid = UID+'_Solo_ECG_Valid.parquet'
    filename_ECG_Timestamp = UID+'_Solo_ECG_Timestamp.parquet'
    if os.path.isfile(os.path.join(path_ECG_Valid,filename_ECG_Valid)):
        print('Loading ECG Valid...')
        start = time.time()
        df_ECG_Valid = pd.read_parquet(os.path.join(path_ECG_Valid,filename_ECG_Valid), engine="fastparquet")
        end = time.time()
        print('Loaded ECG Valid, spent',end - start,'sec for size',df_ECG_Valid.shape)
        print('Loading ECG Timestamp...')
        start = time.time()
        df_ECG_Timestamp = pd.read_parquet(os.path.join(path_ECG_Timestamp,filename_ECG_Timestamp), engine="fastparquet")
        end = time.time()
        print('Load ECG Timestamp spent',end - start,'sec for size',df_ECG_Valid.shape)
        
        ## Another for loop
        column_name = df_ECG_Valid.columns # File name.
        flag_new_UID = True
        total_rows = len(column_name)
        for rr, this_column_name in enumerate(column_name):
        # if True:
        #     rr = 10
        #     this_column_name = column_name[rr]
            print(rr,'/',total_rows,':',this_column_name)
            this_ECG_Valid = df_ECG_Valid[this_column_name].tolist() 
            this_ECG_Timestamp = df_ECG_Timestamp[this_column_name].tolist() # Same file name.
            
            # to remove None values in list
            this_ECG_Valid_clean = list(filter(lambda item: item is not None, this_ECG_Valid))
            this_ECG_Timestamp_clean = list(filter(lambda item: not np.isnan(item), this_ECG_Timestamp))
            
            diff_Timestamp = np.diff(this_ECG_Timestamp_clean)
            
            # For a list: diff_Timestamp.insert(0,diff_Timestamp[0])
            if len(diff_Timestamp) > 0:
                diff_Timestamp_new = np.insert(diff_Timestamp, [0], diff_Timestamp[0], axis=0)
                invalid_diff_Timestamp = [diff_Timestamp_new[idx] for idx,vv in enumerate(this_ECG_Valid_clean) if vv == 'F']
                total_invalid_time = np.sum(invalid_diff_Timestamp) # Accurate time.
                
                count_Valid = this_ECG_Valid_clean.count('T')
                count_Invalid = this_ECG_Valid_clean.count('F')
                
                count_Invalid_time = count_Invalid/fs_ECG # Guess the sampling frequency is always 250 Hz.
            else:
                total_invalid_time = None
                count_Invalid_time = None
                
                
            # Create a dataframe with same column name, put the sum of invalid samples in there.
            if flag_new_UID:
                df_ECG_Valid_time = pd.DataFrame()
                df_ECG_Valid_time[this_column_name] = [total_invalid_time]
                flag_new_UID = False
            else:
                df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
        
                
        
        print('Finished count time for UID',UID)
        filename_output = UID+'_Invalid_count_time.parquet'
        df_ECG_Valid_time.to_parquet(os.path.join(path_output,filename_output), compression=None)
        print('Saved DataFrame to:',os.path.join(path_output,filename_output))

temp_UIDs: ['119', '120']
Loading ECG Valid...
Loaded ECG Valid, spent 1.4054126739501953 sec for size (10000, 1442)
Loading ECG Timestamp...
Load ECG Timestamp spent 1.678910255432129 sec for size (10000, 1442)
0 / 1442 : 119_2021_08_06_19_26_57_ppg_0000.txt
1 / 1442 : 119_2021_08_06_19_26_57_ppg_0001.txt
2 / 1442 : 119_2021_08_12_09_16_02_ppg_0000.txt
3 / 1442 : 119_2021_08_12_09_16_02_ppg_0001.txt
4 / 1442 : 119_2021_08_12_09_16_02_ppg_0002.txt
5 / 1442 : 119_2021_08_12_09_16_02_ppg_0003.txt
6 / 1442 : 119_2021_08_12_09_16_02_ppg_0004.txt
7 / 1442 : 119_2021_08_12_09_16_02_ppg_0005.txt
8 / 1442 : 119_2021_08_12_09_16_02_ppg_0006.txt
9 / 1442 : 119_2021_08_12_09_16_02_ppg_0007.txt
10 / 1442 : 119_2021_08_12_09_16_02_ppg_0008.txt
11 / 1442 : 119_2021_08_12_09_16_02_ppg_0009.txt
12 / 1442 : 119_2021_08_12_09_26_06_ppg_0000.txt
13 / 1442 : 119_2021_08_12_09_26_06_ppg_0001.txt
14 / 1442 : 119_2021_08_12_09_26_06_ppg_0002.txt
15 / 1442 : 119_2021_08_12_09_26_06_ppg_0003.txt
16 / 1442 : 11

/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

109 / 1442 : 119_2021_08_12_10_56_35_ppg_0007.txt
110 / 1442 : 119_2021_08_12_10_56_35_ppg_0008.txt
111 / 1442 : 119_2021_08_12_10_56_35_ppg_0009.txt
112 / 1442 : 119_2021_08_12_11_06_38_ppg_0000.txt
113 / 1442 : 119_2021_08_12_11_06_38_ppg_0001.txt
114 / 1442 : 119_2021_08_12_11_06_38_ppg_0002.txt
115 / 1442 : 119_2021_08_12_11_06_38_ppg_0003.txt
116 / 1442 : 119_2021_08_12_11_06_38_ppg_0004.txt
117 / 1442 : 119_2021_08_12_11_06_38_ppg_0005.txt
118 / 1442 : 119_2021_08_12_11_06_38_ppg_0006.txt
119 / 1442 : 119_2021_08_12_11_06_38_ppg_0007.txt
120 / 1442 : 119_2021_08_12_11_06_38_ppg_0008.txt
121 / 1442 : 119_2021_08_12_11_06_38_ppg_0009.txt
122 / 1442 : 119_2021_08_12_11_16_41_ppg_0000.txt
123 / 1442 : 119_2021_08_12_11_16_41_ppg_0001.txt
124 / 1442 : 119_2021_08_12_11_16_41_ppg_0002.txt
125 / 1442 : 119_2021_08_12_11_16_41_ppg_0003.txt
126 / 1442 : 119_2021_08_12_11_16_41_ppg_0004.txt
127 / 1442 : 119_2021_08_12_11_16_41_ppg_0005.txt
128 / 1442 : 119_2021_08_12_11_16_41_ppg_0006.txt


/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

152 / 1442 : 119_2021_08_12_11_46_51_ppg_0000.txt
153 / 1442 : 119_2021_08_12_11_46_51_ppg_0001.txt
154 / 1442 : 119_2021_08_12_11_46_51_ppg_0002.txt
155 / 1442 : 119_2021_08_12_11_46_51_ppg_0003.txt
156 / 1442 : 119_2021_08_12_11_46_51_ppg_0004.txt
157 / 1442 : 119_2021_08_12_11_46_51_ppg_0005.txt
158 / 1442 : 119_2021_08_12_11_46_51_ppg_0006.txt
159 / 1442 : 119_2021_08_12_11_46_51_ppg_0007.txt
160 / 1442 : 119_2021_08_12_11_46_51_ppg_0008.txt
161 / 1442 : 119_2021_08_12_11_46_51_ppg_0009.txt
162 / 1442 : 119_2021_08_12_11_56_54_ppg_0000.txt
163 / 1442 : 119_2021_08_12_11_56_54_ppg_0001.txt
164 / 1442 : 119_2021_08_12_11_56_54_ppg_0002.txt
165 / 1442 : 119_2021_08_12_11_56_54_ppg_0003.txt
166 / 1442 : 119_2021_08_12_11_56_54_ppg_0004.txt
167 / 1442 : 119_2021_08_12_11_56_54_ppg_0005.txt
168 / 1442 : 119_2021_08_12_11_56_54_ppg_0006.txt
169 / 1442 : 119_2021_08_12_11_56_54_ppg_0007.txt
170 / 1442 : 119_2021_08_12_11_56_54_ppg_0008.txt
171 / 1442 : 119_2021_08_12_11_56_54_ppg_0009.txt


...

/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

1405 / 1442 : 119_2021_08_13_08_43_27_ppg_0003.txt
1406 / 1442 : 119_2021_08_13_08_43_27_ppg_0004.txt
1407 / 1442 : 119_2021_08_13_08_43_27_ppg_0005.txt
1408 / 1442 : 119_2021_08_13_08_43_27_ppg_0006.txt
1409 / 1442 : 119_2021_08_13_08_43_27_ppg_0007.txt
1410 / 1442 : 119_2021_08_13_08_43_27_ppg_0008.txt
1411 / 1442 : 119_2021_08_13_08_43_27_ppg_0009.txt
1412 / 1442 : 119_2021_08_13_08_53_30_ppg_0000.txt
1413 / 1442 : 119_2021_08_13_08_53_30_ppg_0001.txt
1414 / 1442 : 119_2021_08_13_08_53_30_ppg_0002.txt
1415 / 1442 : 119_2021_08_13_08_53_30_ppg_0003.txt
1416 / 1442 : 119_2021_08_13_08_53_30_ppg_0004.txt
1417 / 1442 : 119_2021_08_13_08_53_30_ppg_0005.txt
1418 / 1442 : 119_2021_08_13_08_53_30_ppg_0006.txt
1419 / 1442 : 119_2021_08_13_08_53_30_ppg_0007.txt
1420 / 1442 : 119_2021_08_13_08_53_30_ppg_0008.txt
1421 / 1442 : 119_2021_08_13_08_53_30_ppg_0009.txt
1422 / 1442 : 119_2021_08_13_09_03_33_ppg_0000.txt
1423 / 1442 : 119_2021_08_13_09_03_33_ppg_0001.txt
1424 / 1442 : 119_2021_08_13_09

/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ECG_Valid_time.insert(rr, this_column_name, [total_invalid_time], allow_duplicates=False)
/tmp/ipykernel_1004/3496454500.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

1440 / 1442 : 119_2021_08_13_09_13_36_ppg_0008.txt
1441 / 1442 : 119_2021_08_13_09_13_36_ppg_0009.txt
Finished count time for UID 119
Saved DataFrame to: /mnt/r/ENGR_Chon/NIH_Pulsewatch_Database/Parquet_Files/Solo_ECG_Invalid_count_time/119_Invalid_count_time.parquet
Loading ECG Valid...
Loaded ECG Valid, spent 8.45292067527771 sec for size (10000, 10531)
Loading ECG Timestamp...


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/r/ENGR_Chon/NIH_Pulsewatch_Database/Parquet_Files/Solo_ECG_Timestamp_each_UID/120_Solo_ECG_Timestamp.parquet'

['119', '120']